In [1]:
import pandas as pd
import os
import dask
from dask.diagnostics import ProgressBar
import utils

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

0it [00:00, ?it/s]


In [2]:
csv_ls = [os.path.join('chunk_data/',i) for i in os.listdir('chunk_data/') if ('.csv' in i) and ('-' in i)]

In [4]:
container = []

for i in csv_ls : 
    load = dask.delayed(pd.read_csv)(i)
    container.append(load)
concat_df = dask.delayed(pd.concat)(container,ignore_index=True)

In [5]:
with ProgressBar() : 
    df = concat_df.compute()

[########################################] | 100% Completed |  2.2s


In [6]:
df.columns = ['path','text']

In [7]:
def finer_lemmatize(word) : 
    word = lemmatizer.lemmatize(word,'n')
    word = lemmatizer.lemmatize(word,'v')
    return word

In [14]:
df['lemmatized_text'] = \
df.text.str.split(" ").apply(lambda sent : ','.join(list(map(finer_lemmatize,sent))).replace(",",' '))

In [15]:
df.to_csv('data/summarized_with_path.csv')

___________________

In [16]:
# df = pd.read_csv('data/summarized_with_path.csv')
result_df = pd.DataFrame()
chunk_ls = utils.make_chunk(df.index.tolist(),10)

In [17]:
import importlib
importlib.reload(utils)

<module 'utils' from '/gpfs/haaforfs/home/hyunsikkim/HF_EDGAR_SENTIMENT/utils.py'>

In [ ]:
for idx,index in enumerate(chunk_ls) : 
    tmp = df.iloc[index]
#     print("{} th dataframe is manipulated".format(idx),end='  ')
    %time tmp['parsed_text'] = tmp.lemmatized_text.\
    progress_apply(utils.parsing)
#     print('{} th parsing is finished'.format(idx),end='  ')
    tmp['length'] = tmp.parsed_text.apply(lambda x : len(x.split(" ")))
    use_tmp = tmp[tmp.length <= 100].reset_index(drop=True)
    result_df = pd.concat([result_df,use_tmp],ignore_index=True)
    print(result_df.shape)

In [19]:
result_df.to_csv("data/result_df.csv",index=False)

# Tuning the dictionary list

In [34]:
lm_dict = pd.read_pickle("/mnt/processed/edgar_footnote_partition_indexing/word_type/word_type.pkl")

In [35]:
lm_dict.keys()

dict_keys(['litigious', 'generic', 'positive', 'uncertainty', 'negative', 'constraining', 'genericlong', 'modal'])

In [38]:
pos_ls = [finer_lemmatize(i) for i in lm_dict['positive']]
neg_ls = [finer_lemmatize(i) for i in lm_dict['negative']]

In [39]:
pd.to_pickle(pos_ls,'pos_ls.pkl')
pd.to_pickle(neg_ls,'neg_ls.pkl')

In [5]:
full_dict = pd.read_pickle("/mnt/processed/hf_news_sentiment/Ravenpack_FULL/ravenpack_uni_sent_dict.pkl")

In [20]:
full_dict = pd.read_pickle("/mnt/processed/hf_news_sentiment/BBtn_D0_0_500/BBtn_D0_0_500.pkl")

In [23]:
pos_ls = []
neg_ls = []

for key,val in full_dict.items() : 
    if val < 0 : 
        neg_ls.append(key)
    else : 
        pos_ls.append(key)

In [24]:
pos_ls = [finer_lemmatize(i) for i in pos_ls]
neg_ls = [finer_lemmatize(i) for i in neg_ls]

In [25]:
pd.to_pickle(pos_ls,'d0_pos_ls.pkl')
pd.to_pickle(neg_ls,'d0_neg_ls.pkl')